### Collaborative Filtering
-----------------------------------------------------------------------
----------------------------------------------------------------------

    Collaborative Filtering techniques make recommendations for a user based on ratings and preferences data of many users. The main underlying idea is that if two users have both rated 2 items similarly, then the items that one user has liked that the other user has not yet 

##### In this notebook we will apply user user collaborative filtering technique!

#### User User collaborative filtering !!
------------------------------------------

    This approach relies on the idea that users who have similar rating behaviours so far, share the same tastes and will likely exhibit similar rating behaviours going forward. The algorithm first computes the similarity between users by considering ratings both users have in common

From notebook 2, we are taking similarity funcition. We will use this function to find similarity between active user  and other user using normalized data

In [1]:
def weight_factor(x, y):
    ''' 
    Weight factor implies relationship between user x and user y
    Also know as similarity between user x and user y
    We are using Pearson correlation coefficient here. 
    '''    
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        t1+=i*j
        t2+=i*i
        t3+=j*j
    return t1/(np.sqrt(t2) * np.sqrt(t3))

#### Selecting normalized data from database!

In [2]:
import numpy as np
import pandas as pd

normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')

ratings_df = pd.read_csv('./data/jester_jokes_rating.csv')

In [3]:
normalized_ratings_df.head(2)

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,...,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,100
0,1,74,-4.388108,12.221892,-6.228108,-4.728108,-4.088108,-5.068108,-6.418108,7.601892,...,6.251892,0.0000,0.0000,0.0000,0.0000,0.0000,-2.198108,0.0000,0.0000,0.0000
1,2,100,1.333700,-3.036300,3.613700,1.623700,-5.126300,-12.406300,-3.476300,-8.086300,...,0.073700,-7.6963,-3.0363,5.1137,-2.9363,-4.8863,0.313700,-2.4063,-7.0663,-1.6763


#### Separating normalized ratings dataframe!

We will be separating normalized ratings dataframe into 2 parts, 

    1> Complete ratings: Those users who have rated all 100 jokes
    2> Sparse ratings: Those users who haven't rated all 100 jokes
    
For ease in computation, we will only select active user from sparse ratings, and other users from compete rating groups

In [4]:
# We will be using users who have rated all the 100 jokes as other users.
complete_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] == 100]
print('total user count who have rated all the jokes: ', len(complete_ratings))
# We will be randomly using one out of these users as active user and use it to find 
# similarity with complete_ratings dataset. 
sparse_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] != 100]
print('total user count who have not rated all the jokes: ', len(sparse_ratings))

total user count who have rated all the jokes:  7200
total user count who have not rated all the jokes:  17783


    Selecting one of the user from sparse_ratings matrix for whom the recommendation will be given. We will call him our active user and generate recommendation for him!

In [5]:
# selecting a random user say 1000th user in sparse_ratings list
n = 1000
active_user_id = sparse_ratings.iloc[n, 0]
print("Let's selct a random user with user id {} as active user for\
 which we will recommend the joke".format(str(active_user_id)))

Let's selct a random user with user id 1352 as active user for which we will recommend the joke


In [6]:
print('ratings given by active user {} for 100 jokes'.format(str(active_user_id)))
active_user = sparse_ratings[sparse_ratings['user_id'] == active_user_id]
active_user_rating = active_user.iloc[:, 2:]
active_user_rating

ratings given by active user 1352 for 100 jokes


,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,...,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,100
1351,7.871096,-1.888904,-2.568904,-1.118904,2.821096,-4.078904,-0.188904,1.891096,-2.568904,3.931096,...,0.0,-3.448904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Finding similarity between active user and complete rated user!

In [7]:
# saving active user ratings into 1 d list
active_user_rating_list = active_user_rating.values.ravel()
# finding similarity between active user and all its neighbours among complete rating users
similarity = np.array([(complete_ratings.iloc[i, 0],\
             weight_factor(active_user_rating_list, complete_ratings.iloc[i, 2:]))\
             for i in range(complete_ratings.shape[0])])

#### Sorting the neighbours using similarity 

In [8]:
ind = np.argsort( similarity[:,1] )
similarity = similarity[ind]
similarity

array([[ 7.33800000e+03, -2.87912874e-01],
       [ 1.66100000e+03, -2.87005484e-01],
       [ 1.41410000e+04, -2.70326403e-01],
       ...,
       [ 1.92170000e+04,  3.99469178e-01],
       [ 1.06150000e+04,  4.06355890e-01],
       [ 7.49200000e+03,  4.59458050e-01]])

### Now we have similarity matrix, our next task is to select neighbourhood. 
##### There are few methods to select neighbours:
    1. Use all neighbours in samll dataset
    2. Threshold similarity or distance:
        eg. all users with similarity of 0.1 or above to user for whom we are generating the recommendations will be considered
    3. Random neighbour:
        Can be useful for very large dataset in conjucture with other technique. for eg. randomly select 10,000 neighbour and then threshold them or pick top n
    4. Top n neighbour by similarity or distance
        
### How many neighbours?
##### Between 25 to 100 is ofen used.
        
#### Our method !
    We will be use threshold method followed by random n samples, (where n can be any random number between 25 - 100) but for simplicity we will take n as 30. 
    For threshold we will select other users with similarity > 0.1
    

In [9]:
neighbours = similarity[similarity[:,1] > 0.1]
print('We have {} potential neighbours! Now we will be randomly selecting 30 samples out of them'.format(len(neighbours)))

We have 2323 potential neighbours! Now we will be randomly selecting 30 samples out of them


#### Randomly selecting 30 neighbours

In [10]:
# by replace = False, ensuring that no duplicate neighbour is selected !!
index_30_neighbour = np.random.choice(range(len(neighbours)), 30, replace=False)
selected_neighbours = neighbours[index_30_neighbour]
selected_neighbours

array([[5.03300000e+03, 1.38115736e-01],
       [2.48600000e+04, 2.01804915e-01],
       [8.79200000e+03, 1.64275692e-01],
       [1.80550000e+04, 1.68331699e-01],
       [2.42920000e+04, 2.97820661e-01],
       [2.31690000e+04, 1.27519772e-01],
       [9.91000000e+03, 1.73616580e-01],
       [1.23230000e+04, 1.05936126e-01],
       [1.43930000e+04, 1.49642903e-01],
       [4.13600000e+03, 2.21970162e-01],
       [7.75900000e+03, 1.67753275e-01],
       [1.97550000e+04, 2.47730398e-01],
       [1.46390000e+04, 1.40033820e-01],
       [1.25640000e+04, 2.31544144e-01],
       [4.40000000e+01, 1.18777559e-01],
       [1.22200000e+04, 2.23824961e-01],
       [1.66050000e+04, 2.14481874e-01],
       [2.12570000e+04, 1.05734851e-01],
       [6.95000000e+02, 1.19769265e-01],
       [4.16500000e+03, 1.68320494e-01],
       [1.67740000e+04, 1.33864630e-01],
       [1.92090000e+04, 1.30506564e-01],
       [6.99200000e+03, 1.23297121e-01],
       [1.83380000e+04, 1.84401792e-01],
       [4.935000

    Once we have selected 30 closest neighbours, our task is to create scoring for the jokes !! We will crate scoring for only those jokes which isn't rated yet by the active user yet. 

In [11]:
recommendation_columns = [column for column in active_user_rating.columns if active_user_rating[column].values[0] == 0]

##### Mean value of ratings of active user!!

    The mean value of rating is used to create so as to compensate for the offset created when data was normalized.
    By adding mean value to the predicted score, it will give the probable score the user will give to a joke!

In [12]:
active_user_raw_ratings = ratings_df[ratings_df['user_id'] == active_user_id].iloc[:, 2:]
active_user_mean_rating = np.mean(active_user_raw_ratings.drop(recommendation_columns, axis = 1).values)
active_user_mean_rating

0.6289041095890411

In [13]:
# Selecting neighbours user_id
neighbour_user_id = selected_neighbours[:, 0]


# selectig neighbour user similarity
neighbour_user_similarity = selected_neighbours[:, 1]

# viewing all neighbours user id and similarity
print('neighbours user id: ', neighbour_user_id, '\n\n')
print('neighbours user similarity: ', neighbour_user_similarity)

neighbours user id:  [ 5033. 24860.  8792. 18055. 24292. 23169.  9910. 12323. 14393.  4136.
  7759. 19755. 14639. 12564.    44. 12220. 16605. 21257.   695.  4165.
 16774. 19209.  6992. 18338.  4935. 19156. 23776.  1992. 20513.  3242.] 


neighbours user similarity:  [0.13811574 0.20180492 0.16427569 0.1683317  0.29782066 0.12751977
 0.17361658 0.10593613 0.1496429  0.22197016 0.16775327 0.2477304
 0.14003382 0.23154414 0.11877756 0.22382496 0.21448187 0.10573485
 0.11976927 0.16832049 0.13386463 0.13050656 0.12329712 0.18440179
 0.15712255 0.11302688 0.10520914 0.11078623 0.11091201 0.11256418]


#### Selecting all the data of neighbour!

In [14]:
neighbours_df = complete_ratings[complete_ratings['user_id'].isin(neighbour_user_id)]
len(neighbours_df)

30

In [15]:
# selecting only recommendation columns
print('We will be suggesting one out of {} jokes to the active user \n\n'.format(len(recommendation_columns)))

neighbours_df = neighbours_df[recommendation_columns]
neighbours_df.head()

We will be suggesting one out of 27 jokes to the active user 




,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_80,joke_82,...,joke_90,joke_91,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,100
43,-2.8771,2.0729,2.0729,-2.1071,0.3729,5.3729,-3.6071,-3.8071,-2.8371,0.6629,...,-6.2771,-1.4271,-1.4771,-7.0571,-4.4871,-4.6271,-3.1271,-1.3271,-4.2371,6.7329
694,-2.6178,3.8822,6.8422,1.8022,-0.8678,7.0922,-1.0678,5.1522,7.5722,-4.4178,...,5.5822,6.8922,-4.1778,1.4622,5.2422,-0.9178,-8.2978,-7.6178,-1.5478,0.0522
1991,7.6162,0.6262,0.2862,0.5762,1.1562,0.3862,1.2562,1.5462,-0.5838,1.7462,...,1.0662,1.8862,5.5262,5.5762,-6.3138,0.0962,1.2562,1.9362,1.2062,1.1162
3241,-1.1696,0.5804,-0.2496,-0.7796,-0.8296,0.7704,1.8904,5.9204,-0.9796,-1.7096,...,-1.6096,3.6304,4.6504,-2.2896,-2.4296,-2.6796,-1.6596,2.1804,-0.9296,-0.9296
4135,-3.2266,0.5134,2.1634,-3.1766,1.1934,3.0334,3.5734,1.3434,-1.6666,2.6934,...,-0.4066,-0.6966,-1.9166,-3.4666,-2.6466,3.0834,1.2934,-0.8466,-3.9066,-1.7166


#### checking score for item 0 of the recommendation_columns

In [16]:
item_id = recommendation_columns[0]
print( 'item 0 for which we calculate the score is', item_id)

def score_user_item(item_id, neighbours_df,neighbour_user_similarity, active_user_mean_rating ):
    item_rating = neighbours_df[item_id]
    t1, t2 = 0, 0
    for similarity, norm_rating in zip(neighbour_user_similarity, item_rating):
        t1+= norm_rating * similarity
        t2+= similarity
    score = (t1 + active_user_mean_rating)/t2
    return score

# checking score for a particular joke!
score_user_item(item_id, neighbours_df,neighbour_user_similarity, active_user_mean_rating )
        

item 0 for which we calculate the score is joke_71


-1.8033373756460684

#### suggesting use the joke which has the highesht score among all

In [17]:
# Computing user item score !
top_score = -np.inf
joke_to_suggest = ''

for column in neighbours_df.columns:
    score =score_user_item(column, neighbours_df,neighbour_user_similarity, active_user_mean_rating)
    if score > top_score:
        top_score = score
        joke_to_suggest = column
print('highest score is', top_score)
print('The highest score obtained by the joke among all the unseen jokes is', joke_to_suggest, )    

highest score is 3.9938369096759097
The highest score obtained by the joke among all the unseen jokes is joke_89
